In [1]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import pathlib as pl
from subjetividades import *
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from wordcloud import WordCloud
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from pathlib import Path
from collections import Counter
import numpy as np
from sklearn.cluster import KMeans
import plotly.express as px
from sklearn.manifold import TSNE

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hamso\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
current_path    = pl.Path.cwd().parent
output_folders  = create_output_directories(current_path)
data_folders    = create_data_directories(output_folders[0])
docs_list       = list(data_folders[0].glob('*.xlsx'))
images_folder   = output_folders[1].joinpath('images')
images_folder.mkdir(exist_ok=True)

In [3]:
docs_list[0]

WindowsPath('d:/Repositorios/Vanessa - subjetividades/subjetividades-politicas/data/raw/2. filtrada respuestas_prediccion_completo.xlsx')

In [4]:
df = pd.read_excel(docs_list[0])
df.head()

,NOMBRE,P3,Prediccion a la pregunta P3,Subcategoria a la pregunta P3,Subsubcategoria (IDENTIDAD SOCIAL) a la pregunta P3,P4,Prediccion a la pregunta P4,Subcategoria a la pregunta P4,Subsubcategoria (IDENTIDAD SOCIAL) a la pregunta P4,P5,...,Subcategoria a la pregunta P41,Subsubcategoria (IDENTIDAD SOCIAL) a la pregunta P41,P42,Prediccion a la pregunta P42,Subcategoria a la pregunta P42,Subsubcategoria (IDENTIDAD SOCIAL) a la pregunta P42,P43,Prediccion a la pregunta P43,Subcategoria a la pregunta P43,Subsubcategoria (IDENTIDAD SOCIAL) a la pregunta P43
0,Brandon Santos,"Total, total. Cada... Es evidente, y se siente...",POLARIZACIÓN,NaN,NaN,"Bueno, pues, ¿qué considero yo? Hombre, la pol...",POLARIZACIÓN,NaN,NaN,"""Hombre, la política está muy polarizada hoy e...",...,NaN,NaN,NaN,NaN,NaN,NaN,"No, obviamente en el de Uribe. Pero en la actu...",POLARIZACIÓN,NaN,NaN
1,Michael Patton,¿La polarización? En eso estamos desde hace má...,POLARIZACIÓN,NaN,NaN,En eso estamos desde hace más de diez años. De...,POLARIZACIÓN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,¿La polarización? En eso estamos desde hace má...,POLARIZACIÓN,NaN,NaN
2,Dennis Kennedy,NaN,NaN,NaN,NaN,"En estos gobiernos que has mencionado, ¿cómo m...",EMOCIONES POLÍTICAS,PRIMARIA,NaN,en este oportunidad si escuche un discurso y d...,...,NaN,NaN,"No me considero de ninguna, pues en algún mome...",IDENTIDAD,INDIVIDUAL,NaN,"no, para nada, eso es lo que nos quieren hacer...",POLARIZACIÓN,NaN,NaN
3,Wanda Gray,Hay unas recámaras que resuenan con una inform...,POLARIZACIÓN,NaN,NaN,"La salud más bien mal. Bueno, más bien no se h...",EMOCIONES POLÍTICAS,PRIMARIA,NaN,NaN,...,SOCIAL,REAL,Es difícil llegar a un punto medio otra vez. P...,POLARIZACIÓN,NaN,NaN,Y es impresionante la cantidad de personas que...,POLARIZACIÓN,NaN,NaN
4,Anna Roy,"no, está muy polarizado y yo en lo que en lo q...",POLARIZACIÓN,NaN,NaN,NaN,NaN,NaN,NaN,"Yo sería partidario, como le digo, de que ese ...",...,SOCIAL,REAL,NaN,NaN,NaN,NaN,está muy polarizado y yo en lo que en lo que l...,POLARIZACIÓN,NaN,NaN


In [5]:
lista = []
for i in range(1,45):
    lista.append(f'P{i}')

lista.remove('P1')
lista.remove('P2')
lista.remove('P9')
lista.remove('P10')
lista.remove('P19')
lista.remove('P23')
lista.remove('P24')
lista.remove('P29')
lista.remove('P30')
lista.remove('P37')
lista.remove('P38')
lista.remove('P44')

lista

['P3',
 'P4',
 'P5',
 'P6',
 'P7',
 'P8',
 'P11',
 'P12',
 'P13',
 'P14',
 'P15',
 'P16',
 'P17',
 'P18',
 'P20',
 'P21',
 'P22',
 'P25',
 'P26',
 'P27',
 'P28',
 'P31',
 'P32',
 'P33',
 'P34',
 'P35',
 'P36',
 'P39',
 'P40',
 'P41',
 'P42',
 'P43']

# Semantic Similarity without normalizing the text with stopwords

## Embeddings

In [6]:
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

client = OpenAI(
    api_key=api_key
)

In [7]:
from openai import OpenAI
client = OpenAI()

def get_embedding(text,question,model="text-embedding-3-small"):
    print(f"Construction for the question {question}")
    print(f"Getting embedding for: {text}")
    return client.embeddings.create(
        input=[text],
        model=model
    ).data[0].embedding

## new column named 'embedding' is created which contains the embeddings of the text

In [9]:
df.head()

,NOMBRE,P3,Prediccion a la pregunta P3,Subcategoria a la pregunta P3,Subsubcategoria (IDENTIDAD SOCIAL) a la pregunta P3,P4,Prediccion a la pregunta P4,Subcategoria a la pregunta P4,Subsubcategoria (IDENTIDAD SOCIAL) a la pregunta P4,P5,...,Subcategoria a la pregunta P41,Subsubcategoria (IDENTIDAD SOCIAL) a la pregunta P41,P42,Prediccion a la pregunta P42,Subcategoria a la pregunta P42,Subsubcategoria (IDENTIDAD SOCIAL) a la pregunta P42,P43,Prediccion a la pregunta P43,Subcategoria a la pregunta P43,Subsubcategoria (IDENTIDAD SOCIAL) a la pregunta P43
0,Brandon Santos,"Total, total. Cada... Es evidente, y se siente...",POLARIZACIÓN,NaN,NaN,"Bueno, pues, ¿qué considero yo? Hombre, la pol...",POLARIZACIÓN,NaN,NaN,"""Hombre, la política está muy polarizada hoy e...",...,NaN,NaN,NaN,NaN,NaN,NaN,"No, obviamente en el de Uribe. Pero en la actu...",POLARIZACIÓN,NaN,NaN
1,Michael Patton,¿La polarización? En eso estamos desde hace má...,POLARIZACIÓN,NaN,NaN,En eso estamos desde hace más de diez años. De...,POLARIZACIÓN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,¿La polarización? En eso estamos desde hace má...,POLARIZACIÓN,NaN,NaN
2,Dennis Kennedy,NaN,NaN,NaN,NaN,"En estos gobiernos que has mencionado, ¿cómo m...",EMOCIONES POLÍTICAS,PRIMARIA,NaN,en este oportunidad si escuche un discurso y d...,...,NaN,NaN,"No me considero de ninguna, pues en algún mome...",IDENTIDAD,INDIVIDUAL,NaN,"no, para nada, eso es lo que nos quieren hacer...",POLARIZACIÓN,NaN,NaN
3,Wanda Gray,Hay unas recámaras que resuenan con una inform...,POLARIZACIÓN,NaN,NaN,"La salud más bien mal. Bueno, más bien no se h...",EMOCIONES POLÍTICAS,PRIMARIA,NaN,NaN,...,SOCIAL,REAL,Es difícil llegar a un punto medio otra vez. P...,POLARIZACIÓN,NaN,NaN,Y es impresionante la cantidad de personas que...,POLARIZACIÓN,NaN,NaN
4,Anna Roy,"no, está muy polarizado y yo en lo que en lo q...",POLARIZACIÓN,NaN,NaN,NaN,NaN,NaN,NaN,"Yo sería partidario, como le digo, de que ese ...",...,SOCIAL,REAL,NaN,NaN,NaN,NaN,está muy polarizado y yo en lo que en lo que l...,POLARIZACIÓN,NaN,NaN


In [15]:
for i in lista:
    df_sub = df[[i, 'NOMBRE']]
    df_sub = df_sub.dropna(subset=[i])

    # Apply get_embedding directly instead of using eval()
    df_sub['embedding'] = df_sub[i].apply(lambda x: np.array(get_embedding(x, model="text-embedding-3-small", question=i)))

    # TSNE visualization, important to note that the perplexity is a hyperparameter that can be tuned
    tsne = TSNE(n_components=2, perplexity=df_sub['NOMBRE'].count()-1, random_state=42, init='random', learning_rate=200)
    vis_dims = tsne.fit_transform(np.stack(df_sub['embedding'].values))

    matrix1 = np.vstack(df_sub.embedding.values)
    n_clusters = 4
    kmeans = KMeans(n_clusters = n_clusters, init='k-means++', random_state=42)
    kmeans.fit(matrix1)
    df_sub['Cluster'] = kmeans.labels_

    df_sub['x'] = vis_dims[:, 0]
    df_sub['y'] = vis_dims[:, 1]
    df_sub['Nombre'] = df_sub['NOMBRE']

    fig = px.scatter(df_sub, x='x', y='y',color='Cluster', hover_data=['Nombre'], title=f'Clusters identified visualized in language 2d using t-SNE. Question {i}', labels={'x': 'x', 'y': 'y'}, color_discrete_sequence=px.colors.qualitative.Set1)
    fig.update_traces(text=df_sub['Nombre'], textposition='top center')  # Colocar el nombre en la parte superior del punto
    fig.write_html(images_folder.joinpath(f'tsne_{i}.html'), auto_open=False)
    fig.show()

    df_sub.to_csv(data_folders[0].joinpath(f'Embedded_answers_{i}.csv'))

Construction for the question P3
Getting embedding for: Total, total. Cada... Es evidente, y se siente en todo lado. Se mira por todo lado por las situaciones que se están viviendo, y los dos bandos están peleando fuertemente por mantenerse en el poder y tratan de hacer cosas buenas y malas, como te digo. Pero por hacer esas situaciones, vuelvo y te repito, se están llevando gente inocente que nada tiene que ver con eso, ni, mejor dicho, y en últimas son los que terminan padeciendo.

Construction for the question P3
Getting embedding for: ¿La polarización? En eso estamos desde hace más de diez años. Desde el gobierno Santos
Construction for the question P3
Getting embedding for: Hay unas recámaras que resuenan con una información. Y el oído del individuo se pega a la recámara que se mueve o que vibra en sintonía con alguna cuestión, creencia o algo así. Estas cuestiones de la política están muy vinculadas en términos de las creencias.Estas cuestiones de la política están muy vinculadas

Construction for the question P4
Getting embedding for: Bueno, pues, ¿qué considero yo? Hombre, la política está muy polarizada hoy en día y hay y, pues, es bien sabido por todo, por todos los colombianos que esta polarización tanto de un bando como de otro está perjudicando fuertemente a la población más vulnerable que es la gente pobre que en últimas es la que sufre siempre. En principio pensé que este gobierno pues tenía políticas buenas, puntos buenos, pero como todo, pues, uno se va dando cuenta de que las situaciones también radican en que ellos como tal buscan un beneficio, quieren crear su imagen y aparte de eso pues quieren se nota como las ganas y el deseo de este gobierno de ayudar a la gente pobre pero eh eh, no sé si por por situaciones, pues, de de de los que le siguen en en ya sean los ministerios o o bueno, altos mandos, ya ya en los gabinetes, como que desvirtúan la situación, y bueno, y aparte de eso, pues, también qué pienso yo, mi opinión personal, que los dirigente

Construction for the question P5
Getting embedding for: "Hombre, la política está muy polarizada hoy en día y hay y, pues, es bien sabido por todo, por todos los colombianos que esta polarización tanto de un bando como de otro está perjudicando fuertemente a la población más vulnerable que es la gente pobre que en últimas es la que sufre siempre." Colombia, en sus dos sectores, tanto los de izquierda como los de derecha, si están en el poder, por ejemplo, ahorita la izquierda, los de la derecha buscan destruir en vez de apoyar, en vez de estar. Todo era a la sombra del presidente Uribe. Y en el periodo de Juan Manuel Santos, pues, el hombre se dedicó, pues, a la paz. Pero, pues, realmente sería muy sano y muy saludable para todos que se hicieran las cosas pensando en las cosas buenas y positivas para que todo mejore. Pero, si llegan ellos a ese punto, ese día acabarán con la guerrilla, con los paramilitares, con la delincuencia, porque cada quien trata de subsistir como puede.
Construc

Construction for the question P6
Getting embedding for: No, pues las diferencias son amplias, obviamente, cada cual es que los tiempos también... Uno tiene que ser consecuente con los tiempos porque, por ejemplo, el tiempo de ahorita no es el mismo que el que vivió Iván Duque con el tema de la pandemia y todo eso, pues, bueno, habrá cosas que hicieron positivas, habrá cosas negativas. Cada gobierno tiene algo bueno y, pues, tampoco todo es malo, pero, pues, pienso que, por ejemplo, los grupos al margen de la ley son una farsa. O sea, son integrantes los, por decir así, las guerrillas de las mismas. Son las zonas donde habitan y donde existen esas guerrillas, pero, pues, desafortunadamente la acción o la falta de recursos, no sé, o la falta de, la falta de empuje del gobierno de atacar eso fuertemente, pues, es como complicado. Entonces, la situación es muy complicada y todo el mundo, pues, tiene... O sea... O sea... O sea, la situación es muy complicada y todo el mundo tendrá una razón

Construction for the question P7
Getting embedding for: "Hombre, la política está muy polarizada hoy en día y hay y, pues, es bien sabido por todo, por todos los colombianos que esta polarización tanto de un bando como de otro está perjudicando fuertemente a la población más vulnerable que es la gente pobre que en últimas es la que sufre siempre."  "Entonces, pues, la gente la dejan desbordar y hacen lo que quieren. Por ejemplo, si nos damos de cuenta, por ejemplo, aquí en Cali, en el periodo de Jorge Iván estaba desbordada la delincuencia, estaba desbordada. Por ejemplo, gente que se metía por los carriles, carros, todo eso, inmediatamente tomó el señor Eder el mando. La situación empezó a mejorar y como tal, pues, hay veces que dicen que la... Pues la ley con garrotes que entra, porque la gente, pues desafortunadamente, eso es lo que le gusta, que les impongan las cosas." "Si hubiera más inversión. O bueno, no hay inversión. Que el recurso que hay lo utilizaran adecuadamente, sin tan

...

Construction for the question P39
Getting embedding for: pues mira, bueno si son son son varias cosas no obviamente no es una sola mi papá y la familia de mi padre viene de una posición elevada educada desde por más de un siglo desde 1800 desde mis abuelos bueno han tenido una posición privilegiada pero por el lado de mi mamá han sido fueron campesinos campesinos normales que cultivaban y sembraban café y sembraban frijol y tengo muchos recuerdos de mi abuelo llegando con sus bultos y nos ponía en la finca a desgranar el café y entonces tengo como esas dos es como esas dos imágenes que me han formado eso por un lado por otro lado tuve la fortuna de de estudiar en primero hice el colegio en un colegio privado pero las carreras la hice en una universidad pública entonces he tenido las dos también esas dos visiones y lo otro es posterior a eso durante más de 20 años estuve con un grupo una capaz un programa de capacitación que siempre se enfocaba en el en el emprendimiento en la libre emp

Construction for the question P40
Getting embedding for: Por lo menos poder manifestar esa situación antes te lo acepto. Hace mucho tiempo sí me enganchaba a este tema, era algo que me enganchaba y me daba como rabia y me daba como, sí, como que no, pues, me disgustaba un poco, pero ahorita más bien es como compasión en el hecho de decir, hey, te entiendo, tienes tus argumentos, no van compaginados con los míos, pero no me voy a poner a discutir y no me voy a poner a discernir sobre eso
Construction for the question P40
Getting embedding for: Pues, si estamos hablando de que todo en este momento tiene que ser inclusivo, que tienes que pertenecer a la sociedad si piensas diferente, pues esa persona se debe estar sintiendo fatal, ¿no? Debe estar teniendo de pronto, no sé, negativamente le puede afectar, personalmente le puede afectar, puede crear problemas de ansiedad, de depresión, no sé. O de salud. Podría luchar por sus ideales y causar vandalismo.
Construction for the question P40
Ge

Construction for the question P41
Getting embedding for: Es muy buena, es muy buena, mi papá conmigo, digamos, tenemos una comunicación muy fluida, muy dinámica, él me escucha mucho, digamos que mi mamá también tiene esas características así como izquierdistas, por así decirlo, pero no, es una relación muy buena, hay algo muy especial que se empieza a dar, es que yo tengo dos hermanas, una de mis hermanas cuando se casó se volvió extremadamente religiosa, la religión de ella es una religión cristiana y pues cuando uno está dentro de la religión está dentro de unos marcos de creencia y un poco de cosas, alimentado por estos grupos de pastores que también a la vez se sabe, porque eso ya es conocido. Que esos grupos de pastores están más en el ala conservadora, ahorita los libertarios, hay pastores libertarios y pastores muy conservadores, están metidos en esa ala centro-derechista, por así decirlo, y la discusión se vuelve interesante porque mi hermana ya cuando estábamos pequeños pues c

Construction for the question P42
Getting embedding for: No me considero de ninguna, pues en algún momento pensé que era de centro, pero salió este grupo del Centro democrático, y dije no, no quiero que me relacionen con ese tipo de gente. ele actual gobierno de petro me genera tranquilidad y tengo esperanza. si, pues ellos me mandan informacion por wpp, y yo les leo, pero nada, si ellos me dicen cosas, o me debaten, yo les digo que que bien, jejejeje, si se a el tema debatimos creencias y posturas pero nada más, sin enojarnos. A veces tampoco digo nada. pienso que son personas que tiene una inclinacion politica segun sus experiencias de vida
Construction for the question P42
Getting embedding for: Es difícil llegar a un punto medio otra vez. Pero hay respeto, sí. Hay por lo menos la posibilidad de que uno siente que el interlocutor te está escuchando. A diferencia de que tú estás hablando y yo, no, no, no, no, pero espérate, no, no, no, déjame hablar, déjame hablar, yo quiero...Eso, a

Construction for the question P43
Getting embedding for: No, obviamente en el de Uribe. Pero en la actualidad, Total, total. Cada... Es evidente, y se siente en todo lado. Se mira por todo lado por las situaciones que se están viviendo, y los dos bandos están peleando fuertemente por mantenerse en el poder y tratan de hacer cosas buenas y malas
Construction for the question P43
Getting embedding for: ¿La polarización? En eso estamos desde hace más de diez años. Desde el gobierno Santos. A mi juicio, desde el gobierno Santos empezó la polarización. Eso no es una cosa que empezó grande como llegó. Lo empezó a hacer con el estallido social del año, no recuerdo, hace dos años, en el 2020, o 2020, sí
Construction for the question P43
Getting embedding for: no, para nada, eso es lo que nos quieren hacer ver los medios de comunicación, pero noe s asi, la verdad es que yo hablo con empresarios de derecha y tengo amigos de izquierda, y la verdad es que lo emrpesa uor  o están pensando en estar 

In [17]:
df_sub.embedding[0]

array([-0.01607417, -0.00559705,  0.00014135, ...,  0.01778544,
        0.01346595,  0.02393423])